<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [297]:
import pandas as pd
from collections import Counter
import random
import time
import numpy as np
import math

In [310]:
def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    fin = time.perf_counter()
    return resultado, fin - inicio

def obtener_DataFrame(dict_fpm, P):
  data = {
      "X": list(dict_fpm.keys()),
      "P(X) Real": [P[x] for x in dict_fpm.keys()],
      "Estimación": list(dict_fpm.values()),
      "Diferencia": [abs(P[x] - dict_fpm[x]) for x in dict_fpm.keys()]
  }
  return pd.DataFrame(data)

def estimar_dict_fpm(p, soporte, n):
  """
  Estima la función de probabilidad de masa de una variable aleatoria discreta utilizando un generador de muestras.
  """
  muestras = [p() for _ in range(n)]
  frecuencias = Counter(muestras)

  dict_fpm = {x: frecuencias.get(x, 0) / n for x in soporte}

  return dict_fpm

def calcular_TVD(dict_fpm_estimada, dict_fpm_real):
  """
  Calcula la Distancia Total de Variación (TVD) entre dos distribuciones discretas.
  """
  return 0.5 * sum(abs(dict_fpm_real[x] - dict_fpm_estimada.get(x, 0)) for x in dict_fpm_real)

def calificar_generador(generador_de_muestras, dict_fpm_correcta):
  n = 100_000
  soporte = dict_fpm_correcta.keys()
  dict_fpm_estimada, tiempo = medir_tiempo(lambda: estimar_dict_fpm(generador_de_muestras, soporte, n))
  print(obtener_DataFrame(dict_fpm_estimada, dict_fpm_correcta))
  tvd = calcular_TVD(dict_fpm_estimada, dict_fpm_correcta)
  print()
  print(f"Distancia total de variación (TVD): {tvd:.6f}")
  print(f"Tiempo de ejecución: {tiempo/n:.6f} segundos")


In [319]:
# Implemente dos métodos para generar una binomial Bin(n, p):
N = 10
SOPORTE = list(range(N))
P = 0.3

In [323]:
# Transformada inversa.
def crear_dict_fpm(soporte, p):
  return {v: p(v) for v in soporte}

def calcular_dict_fda(f):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  F = {}
  suma = 0
  for valor, probabilidad in f.items():
    suma += probabilidad
    F[valor] = suma
  return F

def TI(F):
  u = random.random()
  for v, p in F.items():
    if u < p:
      return v

def binomial_fpm(k, n=N, p=P):
    if not 0 <= k <= n:
        return 0.0
    coef = math.comb(n, k)
    prob = coef * (p ** k) * ((1 - p) ** (n - k))
    return prob

fpm = crear_dict_fpm(SOPORTE, binomial_fpm)
fda = calcular_dict_fda(fpm)
generador_de_muestras = lambda: TI(fda)

calificar_generador(generador_de_muestras, fpm)

   X  P(X) Real  Estimación  Diferencia
0  0   0.028248     0.02893    0.000682
1  1   0.121061     0.12129    0.000229
2  2   0.233474     0.23491    0.001436
3  3   0.266828     0.26596    0.000868
4  4   0.200121     0.19773    0.002391
5  5   0.102919     0.10336    0.000441
6  6   0.036757     0.03744    0.000683
7  7   0.009002     0.00874    0.000262
8  8   0.001447     0.00147    0.000023
9  9   0.000138     0.00017    0.000032

Distancia total de variación (TVD): 0.003524
Tiempo de ejecución: 0.000001 segundos


In [322]:
# II) Simulando n ensayos con probabilidad de éxito p y contando el número de éxitos.
def experimento_binomial(n,p):
    exitos = 0
    for ensayo in range(n):
        if random.random() < p:
            exitos += 1
    return exitos

generador_de_muestras = lambda: experimento_binomial(N, P)
calificar_generador(generador_de_muestras, fpm)

   X  P(X) Real  Estimación  Diferencia
0  0   0.028248     0.02813    0.000118
1  1   0.121061     0.12018    0.000881
2  2   0.233474     0.23575    0.002276
3  3   0.266828     0.26658    0.000248
4  4   0.200121     0.19750    0.002621
5  5   0.102919     0.10343    0.000511
6  6   0.036757     0.03742    0.000663
7  7   0.009002     0.00944    0.000438
8  8   0.001447     0.00148    0.000033
9  9   0.000138     0.00009    0.000048

Distancia total de variación (TVD): 0.003918
Tiempo de ejecución: 0.000001 segundos
